In [1]:
# See Chapter 6 Enriching Knowledge Graphs with Data Science

In [4]:
from graphdatascience import GraphDataScience

host = "bolt://localhost:7687"
user = "neo4j"
password = "test1234"

gds = GraphDataScience(host, auth=(user, password), database="neo4j")

In [7]:
! wget https://raw.githubusercontent.com/jbarrasa/gc-2022/main/interop/data/nr-station-links.csv
! wget https://raw.githubusercontent.com/jbarrasa/gc-2022/main/interop/data/nr-stations-all.csv

--2024-02-19 13:44:01--  https://raw.githubusercontent.com/jbarrasa/gc-2022/main/interop/data/nr-station-links.csv
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.110.133, 185.199.111.133, 185.199.108.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.110.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 81618 (80K) [text/plain]
Saving to: ‘nr-station-links.csv.1’

nr-station-links.cs 100%[===================>]  79,71K  --.-KB/s    in 0,03s   

2024-02-19 13:44:01 (3,07 MB/s) - ‘nr-station-links.csv.1’ saved [81618/81618]

--2024-02-19 13:44:01--  https://raw.githubusercontent.com/jbarrasa/gc-2022/main/interop/data/nr-stations-all.csv
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.110.133, 185.199.111.133, 185.199.108.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.110.133|:443... connected.
HTTP request sent, awaiting response... 200

In [14]:
gds.run_cypher(
    """
LOAD CSV WITH HEADERS FROM "file:///nr-stations-all.csv" AS station
CREATE (:Station {name: station.name, crs:station.crs})
    """
)

""


In [18]:
gds.run_cypher(
    """
LOAD CSV WITH HEADERS FROM "file:///nr-station-links.csv" AS track
MATCH (from:Station {crs: track.from})
MATCH (to:Station {crs: track.to})
MERGE (from)-[:TRACK {distance: round(toFloat(track.distance), 2)}]->(to)
    """
)
gds.close()

In [19]:
# Example 6-3. Using the Python API for Graph Data Science to create a graph projection from the railway knowledge graph
from graphdatascience import GraphDataScience

gds = GraphDataScience(host, auth=(user, password), database="neo4j")

gds.graph.project.cypher(
    graph_name='trains',
    node_spec='MATCH (s:Station) RETURN id(s) as id', 
    relationship_spec=
    """
MATCH (s1:Station)-[t:TRACK]->(s2:Station)
RETURN id(s1) AS source, id(s2) as target, t.distance as distance
    """
    )
gds.close()